---

# 🚩 Start (Download files)

In [ ]:
#@markdown ### ▶️ Download
#@markdown Use commas to download multiple files (URL,URL)
urls = "" # @param {type:"string"}

import os

# Create a folder for downloaded files
download_folder = "" # @param {type:"string"}
os.makedirs(download_folder, exist_ok=True)

# Split the string into individual URLs
url_list = urls.split(",")

# Download the files
for url in url_list:
    filename = url.split("/")[-1]  # Get the filename from the URL
    file_path = os.path.join(download_folder, filename)

    # Perform download using wget
    os.system(f'wget -O "{file_path}" "{url.strip()}"')

    print(f"✅ Downloaded: {filename}")

---

In [ ]:
# Upload Files to Hugging Face from Google Colab

# @title Configuration
hf_token = "" # @param {type:"string"}
hf_username = "" # @param {type:"string"}
hf_repo = "" # @param {type:"string"}
local_folder = "/content" # @param {type:"string"}
hf_folder = "" # @param {type:"string"}
repo_type = "model" # @param ["model", "dataset", "space"]
create_repo_if_missing = True # @param {type:"boolean"}
repo_private = True # @param {type:"boolean"}

# Install huggingface_hub if not already installed
try:
    from huggingface_hub import HfApi, login, create_repo
except ImportError:
    print("Installing huggingface_hub...")
    !pip install -q huggingface_hub
    from huggingface_hub import HfApi, login, create_repo

import os
from pathlib import Path

def upload_to_huggingface():
    """Upload files from Colab to Hugging Face repository"""

    # Validate inputs
    if not hf_token:
        print("❌ Error: Hugging Face token is required!")
        print("   Get your token from: https://huggingface.co/settings/tokens")
        return

    if not hf_username or not hf_repo:
        print("❌ Error: Username and repository name are required!")
        return

    if not os.path.exists(local_folder):
        print(f"❌ Error: Local folder '{local_folder}' does not exist!")
        return

    # Check if folder has files
    files = list(Path(local_folder).rglob('*'))
    files = [f for f in files if f.is_file()]
    if not files:
        print(f"❌ Error: No files found in '{local_folder}'!")
        return

    print(f"📊 Found {len(files)} file(s) to upload")

    # Login to Hugging Face
    print("\n🔐 Logging in to Hugging Face...")
    try:
        login(token=hf_token, add_to_git_credential=True)
        print("✅ Successfully logged in!")
    except Exception as e:
        print(f"❌ Login failed: {e}")
        return

    # Initialize API
    api = HfApi(token=hf_token)
    repo_id = f"{hf_username}/{hf_repo}"

    # Check if repo exists
    repo_exists = False
    try:
        api.repo_info(repo_id=repo_id, repo_type=repo_type, token=hf_token)
        print(f"✅ Repository '{repo_id}' found!")
        repo_exists = True
    except Exception as e:
        print(f"⚠️  Repository '{repo_id}' not found or not accessible")

        if create_repo_if_missing:
            print(f"📦 Creating repository '{repo_id}'...")
            try:
                create_repo(
                    repo_id=repo_id,
                    repo_type=repo_type,
                    private=repo_private,
                    token=hf_token,
                    exist_ok=True
                )
                print(f"✅ Repository created!")
                repo_exists = True
            except Exception as e:
                print(f"❌ Failed to create repository: {e}")
                return
        else:
            print("❌ Set 'create_repo_if_missing = True' to create it automatically")
            return

    if not repo_exists:
        print("❌ Cannot proceed without a valid repository")
        return

    # Upload files
    print(f"\n📤 Uploading files from '{local_folder}' to '{repo_id}'...")
    print(f"   Repository type: {repo_type}")
    if hf_folder:
        print(f"   Target folder: {hf_folder}")

    try:
        # Prepare path in repo
        path_in_repo = hf_folder if hf_folder else None

        # Upload folder
        result = api.upload_folder(
            folder_path=local_folder,
            repo_id=repo_id,
            repo_type=repo_type,
            path_in_repo=path_in_repo,
            token=hf_token,
            commit_message=f"Upload files from Colab"
        )

        # Construct URL
        if repo_type == "model":
            base_url = f"https://huggingface.co/{repo_id}"
        elif repo_type == "dataset":
            base_url = f"https://huggingface.co/datasets/{repo_id}"
        else:  # space
            base_url = f"https://huggingface.co/spaces/{repo_id}"

        if hf_folder:
            url = f"{base_url}/tree/main/{hf_folder}"
        else:
            url = f"{base_url}/tree/main"

        print(f"\n✅ Upload completed successfully!")
        print(f"🔗 View your files at: {url}")

    except Exception as e:
        print(f"\n❌ Upload failed: {e}")
        print("\n💡 Troubleshooting tips:")
        print("   1. Make sure your token has 'write' permissions")
        print("   2. Check if the repository exists and you have access")
        print("   3. Verify the repository type is correct (model/dataset/space)")
        print("   4. Try setting 'create_repo_if_missing = True'")

# Run the upload
print("=" * 60)
print("📁 HUGGING FACE UPLOADER")
print("=" * 60)
upload_to_huggingface()

---